In [2]:
import pandas as pd
from glob import glob
from collections import Counter 

BLOOD_MICROBIOME_DIR = '/home/groups/dpwall/briannac/blood_microbiome/'


BAM_MAPPINGS_FILE = '/home/groups/dpwall/briannac/general_data/bam_mappings.csv'
CONC_FILE= '/home/groups/dpwall/briannac/blood_microbiome/data/decontam/concentrations.csv'
NYGC_FILE = '/home/groups/dpwall/briannac/blood_microbiome/intermediate_files/decontam/NYGC_DNA_stats'

ABUNDANCES_ALL = '/home/groups/dpwall/briannac/blood_microbiome/data/kraken_align/microbe_filtered_species.tsv'
IHART_FLAGSTAT_FILE = '/home/groups/dpwall/briannac/blood_microbiome/data/ihart_flagstat.csv'


DECONTAM_OUT_FILE = BLOOD_MICROBIOME_DIR + 'intermediate_files/decontam/for_decontam_microbe.csv'


In [3]:
# Read in bam mappings
bam_mappings = pd.read_csv(BAM_MAPPINGS_FILE, sep='\t', index_col=1)
bam_mappings = bam_mappings[bam_mappings['status']=='Passed_QC_analysis_ready']


In [7]:
# Get concentrations
xls_files = glob(NYGC_FILE + '/*SampleSubmission*.xls') + glob(NYGC_FILE + '/*/*SampleSubmission*.xls')
xlsx_files = glob(NYGC_FILE + '/*/*manifest.*xlsx') + glob(NYGC_FILE + '/*manifest.*xlsx')
xls = pd.concat([pd.read_excel(f) for f in xls_files])
xls.index = xls['Client Sample ID']
xls = xls[['Concentration (ng/µL)']]

def read_xlsx(file_name):
    df = pd.read_excel(file_name,engine='openpyxl')
    df.columns = df.iloc[11]
    df.index = df['UDF/Client Sample ID']
    df.drop('UDF/Client Sample ID', axis=1, inplace=True)
    df = df.iloc[14:][['UDF/Client Sample Concentration']]
    df.columns = ['Concentration (ng/µL)']
    return df

xlsx = pd.concat([read_xlsx(f) for f in xlsx_files])
concentrations = pd.concat([xls, xlsx])
concentrations = concentrations[~pd.isna(concentrations.index)]
concentrations = concentrations[~pd.isna(concentrations['Concentration (ng/µL)'])]
#concentrations.index = [c if c in bam_mappings.index else c.split('_')[0] for c in concentrations.index]
concentrations = concentrations[[c in bam_mappings.index for c in concentrations.index]] # Limit to those in bam file

print(len(concentrations), 'samples')
print(len(set(concentrations.index)), 'unique samples')
concentrations.to_csv(CONC_FILE)

/home/users/briannac/.local/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


3110 samples
3110 unique samples


# Set up for decontam

In [16]:
abundances = pd.read_table(ABUNDANCES_ALL, index_col=0)
concentrations = pd.read_csv(CONC_FILE, index_col=0)

ihart_flagstat = pd.read_csv(IHART_FLAGSTAT_FILE, index_col=0)
ihart_flagstat.index = [i.replace('-LCL', '_LCL') for i in ihart_flagstat.index]

samples = list(set(abundances.columns).intersection(set(abundances.columns)).intersection(
    ihart_flagstat.index).intersection(concentrations.index))
ihart_flagstat = ihart_flagstat.loc[samples]
total_reads = ihart_flagstat['Total_Reads']-ihart_flagstat['Supplementary']-ihart_flagstat['Duplicates']

for_decontam = abundances.transpose()
for_decontam = for_decontam.loc[samples]
for_decontam = for_decontam.apply(lambda x: x/total_reads.values)
for_decontam['concentration'] = (concentrations['Concentration (ng/µL)'].loc[samples].values)#*
#                                 ihart_flagstat['Mapped'].loc[samples].values/
#                                 total_reads.loc[samples].values)
for_decontam['batch'] = ['%s_%s' % (i,j) for i,j in zip(bam_mappings.loc[samples].bio_seq_source.values, bam_mappings.loc[samples].sequencing_plate.values)]
batch_counts = Counter(for_decontam.batch)
for_decontam[[batch_counts[b]>10 for b in for_decontam.batch]].to_csv(DECONTAM_OUT_FILE)